# Fake News Classifier Final Report

## Introduction

## Importing Necessary Libraries

In [ ]:
# libraries for loading + preprocessing
import pandas as np

# libraries for NLTK


# libraries for spaCy


# libraries for Gensim


# libraries for ML models
from scikitlearn.model_selection import train_test_split



In [ ]:
# import dataset
df = pd.read_csv('')

## Exploratory Data Analysis

- look at missing values
- look at size
- look at data type
- look at distribution of type

## Text Cleaning and Preprocessing

- drop na
- lower case
- remove stop words
- 

## Natural Language Processing

### NLTK

### Spacy

tokenize, lemmatize, POS tagging, entity recognition

### Gensim

## Building Classifiers

- decision tree
- knn
- XGBoost
- 

## Evaluating Model Performance

## Model Explainability

- SHAP
- LIME